In [1]:
import random

# Durasi kegiatan
activities = {'A': 2, 'B': 3, 'C': 2, 'D': 4}
activity_list = ['A', 'B', 'C', 'D']

# Fungsi untuk memeriksa syarat
def check_constraints(individual):
    violations = []

    # Syarat 1: A harus sebelum B
    if individual[0] + activities['A'] > individual[1]:
        violations.append("A harus sebelum B")

    # Syarat 2: C dan D tidak boleh bersamaan
    c_start, c_end = individual[2], individual[2] + activities['C']
    d_start, d_end = individual[3], individual[3] + activities['D']
    if not (c_end <= d_start or d_end <= c_start):
        violations.append("C dan D overlap")

    # Syarat 3: Total durasi <= 8 jam
    end_times = [individual[i] + activities[act] for i, act in enumerate(activity_list)]
    if max(end_times) > 8:
        violations.append("Melebihi 8 jam")

    return violations

# Perhitungan fitness
def calculate_fitness(individual):
    violations = check_constraints(individual)
    return max(0, 10 - len(violations))  # Skor fitness lebih tinggi jika sedikit pelanggaran

# Fungsi utama Algoritma Genetika
def genetic_algorithm(population_size=50, generations=50):
    # Inisialisasi populasi dengan waktu acak untuk masing-masing kegiatan
    population = [[random.randint(0, 7) for _ in range(4)] for _ in range(population_size)]
    best_solution = None
    best_fitness = 0

    for generation in range(generations):
        fitness_scores = [calculate_fitness(ind) for ind in population]

        # Temukan yang terbaik di generasi saat ini
        current_best_fitness = max(fitness_scores)
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_solution = population[fitness_scores.index(current_best_fitness)]

        # Seleksi: memilih dua individu terbaik
        selected = [population[i] for i in sorted(range(population_size), key=lambda x: fitness_scores[x], reverse=True)[:2]]

        # Crossover dan mutasi
        new_population = selected.copy()
        while len(new_population) < population_size:
            parent1, parent2 = random.sample(selected, 2)
            point = random.randint(1, 3)  # Titik crossover acak antara 1 dan 3
            child1 = parent1[:point] + parent2[point:]
            child2 = parent2[:point] + parent1[point:]

            # Mutasi: 10% kemungkinan untuk mengubah waktu mulai kegiatan secara acak
            for child in [child1, child2]:
                if random.random() < 0.1:
                    child[random.randint(0, 3)] = random.randint(0, 7)

                # Pastikan bahwa setelah mutasi, solusi tetap valid
                while calculate_fitness(child) == 0:  # Cek apakah solusi valid
                    child[random.randint(0, 3)] = random.randint(0, 7)  # Jika tidak valid, ulangi mutasi

            new_population.extend([child1, child2])

        # Batasi ukuran populasi agar tetap sesuai
        population = new_population[:population_size]

    return best_solution, best_fitness

# Fungsi utama untuk menjalankan GA dan menampilkan hasil
def main():
    # Menjalankan algoritma genetika
    best_solution, best_fitness = genetic_algorithm()

    # Menampilkan solusi terbaik
    print("=== SOLUSI TERBAIK ===")
    if best_solution:
        # Urutkan kegiatan berdasarkan waktu mulai
        sorted_activities = sorted(enumerate(best_solution), key=lambda x: x[1])
        sorted_activity_names = [activity_list[idx] for idx, _ in sorted_activities]

        print(f"Jadwal Kegiatan: {sorted_activity_names}")
        print(f"Kromosom: {best_solution}")
        print(f"Fitness: {best_fitness:.2f}")

        # Tampilkan waktu mulai dan selesai setiap kegiatan sesuai urutan waktu
        for idx, (activity_index, start_time) in enumerate(sorted_activities):
            activity_name = activity_list[activity_index]
            end_time = start_time + activities[activity_name]
            print(f"Kegiatan {activity_name}: Mulai jam {start_time}:00, Selesai jam {end_time}:00")

if __name__ == "__main__":
    main()

=== SOLUSI TERBAIK ===
Jadwal Kegiatan: ['C', 'A', 'D', 'B']
Kromosom: [2, 5, 0, 4]
Fitness: 10.00
Kegiatan C: Mulai jam 0:00, Selesai jam 2:00
Kegiatan A: Mulai jam 2:00, Selesai jam 4:00
Kegiatan D: Mulai jam 4:00, Selesai jam 8:00
Kegiatan B: Mulai jam 5:00, Selesai jam 8:00
